In [1]:
import numpy as np
from prettytable import PrettyTable
import copy

In [2]:
class Cell:
    def __init__(self, values: np.array, vector: np.array):
        self.__values = values
        self.__vector = vector

    def __eq__(self, other):
        if isinstance(other, Cell):
            return np.all(self.values == other.values) and np.all(self.vector == other.vector)
        return False

    def __ge__(self, other):
        if isinstance(other, list):
            return [self.__ge__(i) for i in other]
        return np.all(self.vector.__ge__(other.vector))

    def __add__(self, other):
        return Cell(self.values + other.values, self.vector + other.vector)

    def __str__(self):
        return str(self.values) + " " + str(self.vector)

    def __repr__(self):
        return str(self)

    @property
    def values(self):
        return self.__values

    @values.setter
    def values(self, values: np.array):
        self.__values = values

    @property
    def vector(self):
        return self.__vector

    @vector.setter
    def vector(self, vector: list[int]):
        self.__vector = vector

    @staticmethod
    def init_cells(coefficients: list[list[int]]) -> list["Cell"]:
        cells = []
        n = len(coefficients[0])
        for i, vs in enumerate(zip(*coefficients)):
            vector = np.eye(1, n, i, dtype=np.int32)[0]
            cell = Cell(np.array(vs), vector)
            cells.append(cell)
        return cells

In [3]:
class ResultCell:
    def __init__(self, cell: Cell, arrows: list[int], m: int):
        self.__cell = cell
        self.__arrows = arrows
        self.__m = m

    def __str__(self):
        m_str = ""
        if self.m < 0:
            m_str = "--> s" + str(-self.m)
        elif self.m > 0:
            m_str = ">= s" + str(self.m)
        return str(np.array(self.arrows)) + "\n" + str(self.cell) + "\n" + m_str

    def __repr__(self):
        return str(self)

    @property
    def cell(self):
        return self.__cell

    @cell.setter
    def cell(self, cell: Cell):
        self.__cell = cell

    @property
    def arrows(self):
        return self.__arrows

    @arrows.setter
    def arrows(self, arrows: list[int]):
        self.__arrows = arrows

    @property
    def m(self):
        return self.__m

    @m.setter
    def m(self, m: list[int]):
        self.__m = m

In [4]:
def calc_process_m(cell: Cell, solutions: list[Cell]) -> int:
    if np.all(cell.values == 0):
        if cell not in solutions:
            solutions.append(cell)
        return -len(solutions)
    l = np.where(cell >= solutions)[0]
    m = l[0] + 1 if l.size > 0 else None
    if m is not None:
        return m
    return 0
    

def solve(coefficients: list[list[int]]) -> tuple[list[list[ResultCell]], list[Cell]]:
    solutions = []
    first = Cell.init_cells(coefficients)
    result = [[]]
    for cell in first:
        m = calc_process_m(cell, solutions)
        result[0].append(ResultCell(cell, [], m))
    flag = True
    while flag:
        curr_result = []
        flag = False
        for step_i, result_cell in enumerate(result[-1]):
            if result_cell.m != 0:
                continue
            flag = True
            for first_i, f_cell in enumerate(first):
                if np.dot(result_cell.cell.values, f_cell.values.T) >= 0:
                    continue
                tmp = result_cell.cell + f_cell
                m = calc_process_m(tmp, solutions)
                for i, v in enumerate(curr_result):
                    if tmp == v.cell:
                        curr_result[i].arrows += [step_i + 1]
                        break
                else:
                    curr_result.append(ResultCell(tmp, [step_i + 1], m))
        result.append(curr_result)
    return result[:-1], solutions

In [5]:
def print_result(result: list[list[ResultCell]]):
    result_ = copy.deepcopy(result)
    n = max(map(len, result_))
    for r in result_:
        while len(r) < n:
            r.append("")

    table = PrettyTable()
    table.field_names = range(1, n + 1)
    for r in result_:
        table.add_row(r, divider=True)

    print(table)
    

def print_solutions(solutions: list[Cell]):
    print("Solution vectors:")
    for b_i, b_v in enumerate(solutions):
        print(f"s{b_i + 1} = {b_v.vector}")

In [6]:
def input_():
    n = int(input("n >>"))
    coefficients = []
    for i in range(n):
        coefficients.append(list(map(int, input(f"e{i + 1} >>").split())))
    return coefficients

**Enter the number of equations and their coefficients:**

*Input example:*\
```n >> 2```\
```e1 >> -1 1 2 -3```\
```e2 >> -1 3 -2 -1```

In [7]:
es = input_()

n >> 2
e1 >> -1 1 2 -3
e2 >> -1 3 -2 -1


In [8]:
out_ = solve(es)
print_result(out_[0])

+-------------------+-------------------+-------------------+-------------------+
|         1         |         2         |         3         |         4         |
+-------------------+-------------------+-------------------+-------------------+
|         []        |         []        |         []        |         []        |
| [-1 -1] [1 0 0 0] |  [1 3] [0 1 0 0]  | [ 2 -2] [0 0 1 0] | [-3 -1] [0 0 0 1] |
|                   |                   |                   |                   |
+-------------------+-------------------+-------------------+-------------------+
|       [1 2]       |       [2 3]       |       [2 4]       |       [3 4]       |
|  [0 2] [1 1 0 0]  |  [3 1] [0 1 1 0]  | [-2  2] [0 1 0 1] | [-1 -3] [0 0 1 1] |
|                   |                   |                   |                   |
+-------------------+-------------------+-------------------+-------------------+
|        [1]        |       [1 2]       |        [1]        |      [2 3 4]      |
| [-1  1] [2 1 0

In [9]:
print_solutions(out_[1])

Solution vectors:
s1 = [0 1 1 1]
s2 = [4 2 1 0]


**Handwritten version of the example:**

![](https://i.imgur.com/QzLmL83.png)